# NER Wikidata

## Set up

In [1]:
import pandas as pd
import urllib.request
import re 
import json

In [2]:
df = pd.read_csv("/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank.csv")

## 1. Retrieve entity names

### 1.1 Decode into bytes

In [29]:
df1 = df[:1000]

In [ ]:
df1 = df[:1000]
list1 = df1.Entity.to_list()
Name = []
for i, entity in enumerate(list1):
    print('Entity', i)
    response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids='+entity+'&languages=en&format=json')
    value = response.read().decode()
    try:
        value = value.split('"value":"')[1].split('"}}}}')[0]
        Name.append(value)
    except:
        Name.append('NA')
df1.insert(1, column = 'Name', value = Name)
del i, entity, Name, list1

In [8]:
df1.query('Name == "NA"')

,Entity,Name,QRank
18,Q110918355,NA,34741612


In [6]:

response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids=Q1107&languages=en&format=json')
value = response.read().decode()
print(value)
# value = value.split('"value":"')[1].split('"}}}}')[0]
# Name.append(value)

{"entities":{"Q1107":{"type":"item","id":"Q1107","labels":{"en":{"language":"en","value":"anime"}}}},"success":1}


In [5]:
list1 = df1.Entity.to_list()
list1[999]

'Q1107'

### 1.2 Decode into dictionary

In [ ]:
def top_N_NEs(input_csv_ranking_file_path: str, output_csv_file_path: str, top_N: int = 10):
    df = pd.read_csv(input_csv_ranking_file_path)
    df2 = df[:top_N]
    list2 = df2.Entity.to_list()
    Name = []
    for i, entity in enumerate(list2):
        print('Entity', i)
        response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids='+entity+'&languages=en&format=json')
        value = response.read().decode('utf-8')
        value = json.loads(value)
        print(type(entity))
        try:
            value = value['entities'][entity]['labels']['en']['value']
            # value = value['entities'][f'{entity}]['labels']['en']['value']
            Name.append(value)
        except:
            Name.append('NA')
    df2.insert(1, column = 'Name', value = Name)
    df2.to_csv(output_csv_file_path)
    del i, entity, Name, list2

top_N_NEs("/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank.csv",
          "/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank_top10.csv")

In [23]:
"Q317521" "Q1107"
response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids=Q317521&languages=en&format=json')
value = response.read()
value = json.loads(value.decode('utf-8'))
print(value)
value['entities']['Q317521']['labels']['en']['value']

{'entities': {'Q317521': {'type': 'item', 'id': 'Q317521', 'labels': {'en': {'language': 'en', 'value': 'Elon Musk'}}}}, 'success': 1}


'Elon Musk'

## 2. 2nd two weeks

##### Goal 0: Expand top_n_NEs above to return the top-N NEs as a list of strings

In [ ]:
def top_N_NEs(input_csv_ranking_file_path: str, top_N: int = 10):
    df = pd.read_csv(input_csv_ranking_file_path)
    df2 = df[:top_N]
    list2 = df2.Entity.to_list()
    Name = []
    for i, entity in enumerate(list2):
        print('Entity', i)
        response = urllib.request.urlopen('https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids='+entity+'&languages=en&format=json')
        value = response.read().decode('utf-8')
        value = json.loads(value)
        print(type(entity))
        try:
            value = value['entities'][entity]['labels']['en']['value']
            # value = value['entities'][f'{entity}]['labels']['en']['value']
            Name.append(value)
        except:
            Name.append('NA')
    return Name
    del i, entity, Name, list2

list1 = top_N_NEs("/Users/yangyuting/Library/Mobile Documents/com~apple~CloudDocs/01PhD/02ISU/04Classes/2022Fall/COM S 579X NLP/Project/wikidata-qrank.csv")

##### Goal 1: Understand how to evaluate an NER model/method based on the papers you read in Goal 2 for the first two weeks. 

SOTA performance: F1 score, precision, and recall

### Goal 2: Develop a Python function that takes a sentence as input and returns a NER tag sequence

(in an appropriate tagging schemes, such as BIO), using simple string match pseudocode below. One thing you need to think is how to match overlapping NEs, e.g., “iowa” vs. “iowa state university”. One easy heurstics is to always match the longest one.

[Named Entity Recognition Tagging](https://cs230.stanford.edu/blog/namedentity/)  
[NER model evaluation](https://towardsdatascience.com/entity-level-evaluation-for-ner-task-c21fb3a8edf)

In [ ]:
import re
sen = "I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames."
NE_list = ['Iowa State University', 'Ivy College of Business', 'College of Business','Iowa', 'Ames']

def brutal_force_ENR(sentence:str, NE_list: list[str], scheme='BIO'):
    sen_list = sen.split()
    print(sen_list)
    return_list = ['O']*len(sen_list)
    print('\nThe list to be revised: \n',return_list)
    
    pos_list = []
    for ne in NE_list:
        position = sentence.find(ne)
        print(ne, position)
        l = len(ne.split())
        pos_list.append([l, position])
    print('\n The number of tokens in the entity, the starting position of the entity:\n', pos_list)
    
    str_len = 0
    id_list = []
    for id, token in enumerate(sen_list):
        str_len += len(token) + 1
        for l, pos in pos_list:
            if str_len == pos:
                id_list.append([id+1, l])
    print('\nThe position in the return_list, the number of tokens to be revised\n',id_list)
    
    for id, length in id_list:
        return_list[id] = 'B'
        while length > 1:
            return_list[id+1] = 'I'
            length -= 1
            id += 1
    print('\n Output: \n', sen_list, '\n', return_list)
    return return_list
brutal_force_ENR(sen, NE_list)


# from seqeval.metrics import accuracy_score
# from seqeval.metrics import f1_score
from seqeval.metrics import classification_report
def a_judge_function(predicted_tag_seq: list[list], ground_truth_tag_seq: list[list]):
    return classification_report([ground_truth_tag_seq], [predicted_tag_seq])
    


In [ ]:
a_judge_function(['B','I',"O"], ['B','O', "O"])

### Goal 3

In [61]:
def test_on_CONLL(sentences: list[str], ground_truth_tag_sequences: list[str], NE_list: list[str], scheme="BIO"):
    for (sentence, ground_truth_tag_seq) in zip(sentences, ground_truth_tag_sequences):
        print(sentence, ground_truth_tag_seq)
        tag_seq = brutal_force_ENR(sentence, NE_list, scheme = scheme)
        print(ground_truth_tag_seq, tag_seq)
        performance_matrix = a_judge_function(ground_truth_tag_seq, tag_seq)
    # return len(sentences)
    return performance_matrix, len(sentences)

sen = "I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames."
NE_list = ['Iowa State University', 'Ivy College of Business', 'Iowa', 'Ivy', 'Ames']
true_list = ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'B']
test_on_CONLL([sen], [true_list], NE_list) 

I joined the Ph.D. program of Ivy College of Business at Iowa State University in Ames. ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'B']
['I', 'joined', 'the', 'Ph.D.', 'program', 'of', 'Ivy', 'College', 'of', 'Business', 'at', 'Iowa', 'State', 'University', 'in', 'Ames.']

The list to be revised: 
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Iowa State University 57
Ivy College of Business 30
Iowa 57
Ivy 30
Ames 82

 The number of tokens in the entity, the starting position of the entity:
 [[3, 57], [4, 30], [1, 57], [1, 30], [1, 82]]

The position in the return_list, the number of tokens to be revised
 [[6, 4], [6, 1], [11, 3], [11, 1], [15, 1]]

 Output: 
 ['I', 'joined', 'the', 'Ph.D.', 'program', 'of', 'Ivy', 'College', 'of', 'Business', 'at', 'Iowa', 'State', 'University', 'in', 'Ames.'] 
 ['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'B']
['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I

('              precision    recall  f1-score   support\n\n           _       1.00      1.00      1.00         3\n\n   micro avg       1.00      1.00      1.00         3\n   macro avg       1.00      1.00      1.00         3\nweighted avg       1.00      1.00      1.00         3\n',
 1)

In [ ]:
classification_report([['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'O']], [['O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'I', 'I', 'O', 'B', 'I', 'I', 'O', 'B']])